In [28]:
import swan_vis as swan
import scanpy as sc
import glob
import pandas as pd

In [2]:
annot = 'gencode.vM21.primary_assembly.annotation_UCSC_names.gtf'
gtf1 = 'sc_mb.gtf'
gtf2 = 'sn_mb.gtf'
gtf3 = 'sn_mt.gtf'

sg = swan.SwanGraph(sc=True)
sg.add_annotation(annot, verbose=True)


Adding annotation to the SwanGraph


Processing transcripts: 100%|█████████▉| 141800/141862 [01:02<00:00, 2265.44it/s]


Reindexing and sorting entries on genomic location...


Reindexing edges:  97%|█████████▋| 730000/754537 [00:00<00:00, 793620.52it/s] 



Annotation added to the SwanGraph


In [3]:
sg.add_transcriptome(gtf1, verbose=True)


Adding transcriptome to the SwanGraph


Processing transcripts: 100%|█████████▉| 34000/34097 [02:35<00:00, 218.54it/s] 


Reindexing and sorting entries on genomic location...


Reindexing edges:  97%|█████████▋| 733000/757448 [00:01<00:00, 670332.02it/s]



Transcriptome added to the SwanGraph


In [4]:
sg.add_transcriptome(gtf2, verbose=True)


Adding transcriptome to the SwanGraph


Processing transcripts: 100%|█████████▉| 25100/25193 [02:26<00:00, 171.41it/s]  


Reindexing and sorting entries on genomic location...


Reindexing edges:  97%|█████████▋| 733000/757733 [00:01<00:00, 626922.89it/s]



Transcriptome added to the SwanGraph


In [5]:
sg.add_transcriptome(gtf3, verbose=True)

  0%|          | 0/16662 [00:00<?, ?it/s]


Adding transcriptome to the SwanGraph


Processing transcripts: 100%|█████████▉| 16600/16662 [02:24<00:00, 115.13it/s]  


Reindexing and sorting entries on genomic location...


Reindexing edges:  97%|█████████▋| 733000/757922 [00:01<00:00, 645221.47it/s]



Transcriptome added to the SwanGraph


In [7]:
len(sg.t_df.index)

144358

In [17]:
# concatenate and format the h5ad files
i = 0
for f in glob.glob('*h5ad'):
    exp = f.split('.')[0]
    if i == 0:
        adata = sc.read(f)
        adata.obs['exp'] = exp
    else:
        temp = sc.read(f)
        temp.obs['exp'] = exp
        adata = adata.concatenate(temp)
    i += 1

In [18]:
adata.var.shape

(22806, 3)

In [19]:
adata.X.shape

(464, 22806)

In [21]:
tids = adata.var.transcript_id.tolist()

In [23]:
bcs = adata.obs.index.tolist()

In [25]:
X = adata.X.transpose()

In [26]:
X.shape

(22806, 464)

In [29]:
df = pd.DataFrame(index=tids, columns=bcs, data=X)

In [30]:
df.head()

,ACTATGCAACACGACCGTGCTAGC-0-0,GAGTTAGCTCTTCACATTCCGATC-0-0,CTGGCATAACAAGCTACTTTGGTC-0-0,ATCATTCCGACAGTGCAGCGAAAC-0-0,ATTGGCTCGAGCTGAACTTTGGTC-0-0,CGACTGGATGGTGGTACGTCTAGG-0-0,CGAACTTACTCAATGAGTGCTAGC-0-0,GATAGACAAAGACGGAGTGCTAGC-0-0,GATAGACAATTGGCTCCGCCCGGA-0-0,AAGACGGACACCTTACCACAATTG-0-0,...,TTCACGCAGCCAAGACGTTCAACA-1,AGTGGTCAGGAGAACATTATTCTG-1,AGTCACTAAGATGTACAATTTCTC-1,CAGCGTTACCTAATCCAATTTCTC-1,AGATGTACGGTGCGAAGTTCAACA-1,GCCACATACCTCTATCTACGGTTA-1,ATTGAGGAAACGCTTAAATTTCTC-1,GGAGAACAGCTCGGTATTATTCTG-1,TGGTGGTAACAGCAGATGAAGCAA-1,TCCGTCTAAGGCTAACAATTTCTC-1
ENSMUST00000156816.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000192286.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000195445.1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000027036.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000115529.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df.to_csv('c2c12_abundance.tsv', sep='\t')

In [33]:
adata.obs.to_csv('c2c12_metadata.tsv', sep='\t')